## Create information gain metric structure

## Make structure where infromation gain is easily accessed

In [1]:
%pip install autopep8 networkx obonet pycodestyle
%pip install pandas sklearn ontobio
%pip install matplotlib

  Using cached autopep8-2.0.2-py2.py3-none-any.whl (45 kB)
  Using cached pycodestyle-2.10.0-py2.py3-none-any.whl (41 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import MeanShift, estimate_bandwidth, AffinityPropagation, SpectralClustering, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from ontobio.ontol_factory import OntologyFactory
from ontobio.assoc_factory import AssociationSetFactory
import os
import json
import math
import pickle
import matplotlib.pyplot as plt

In [3]:
FOLDER = "./../data/"

In [4]:
namespace_list = ["cellular_component", "molecular_function", "biological_process"]

# maybe also save the alternative IDs, not sure right now
class Node:
  def __init__(self, name):
    self.name = name
    self.visited = False  # for BFS
    self.reachable_nodes_count = 1  
    self.infromationMetric = 0
    self.namespace = -1
    #without alternative IDs
    self.parents = []  
    self.children = []
    


For now only checking part_of and is_a relationships. not regulates, positively_regulates and negatively_regulates

In [5]:
goFile = open(FOLDER + 'go-basic.obo', 'r')
Lines = goFile.readlines()

nodeDict = dict()
alternative_ID_dict = dict()

count_IS_a_relationship = 0
count_part_of_relationship = 0
lastID  = ""
altID = []



for line in Lines:
    # NEW GO term
    if line[0:4] == "id: ":
        if altID:
            alternative_ID_dict[lastID] = altID
        altID = [] 
        
        lastID = line[4:-1]
        if not (lastID in nodeDict):
            nodeDict[lastID] = Node(lastID)
    # When the lastID was checked to be obsolete, then move on, until there is new lastID
    if lastID == "obsolete":
        continue
    
    # Adding namespace

    if line[0:10] == "namespace:":
        namespace = line.split()[1]
        for i in range(len(namespace_list)):
            if (namespace == namespace_list[i]):
                nodeDict[lastID].namespace = i
                
    # ALternative name for ID
    if line[0:8] == "alt_id: ":
        altID.append(line[8:-1])
    
    # Check if gene is obsolete
    if line[0:11] == "is_obsolete" :
        altID = []
        nodeDict.pop(lastID)
        lastID = "obsolete"
        
    # IS_a relationship    
    if line[0:5] == "is_a:":
        count_IS_a_relationship +=1
        isA = line[5:-1].split()[0]
        nodeDict[lastID].parents.append(isA)  
        if not (isA in nodeDict):
            nodeDict[isA] = Node(isA)
        nodeDict[isA].children.append(lastID)
        
    # part_of relationship    
    if line[0:21] == "relationship: part_of":
        count_part_of_relationship +=1
        partOf = line[21:-1].split()[0]
        nodeDict[lastID].parents.append(partOf)  
        if not (partOf in nodeDict):
            nodeDict[partOf] = Node(partOf)
        nodeDict[partOf].children.append(lastID)
        
    
goFile.close()        

alternative_ID_dict[lastID] = altID
print(count_IS_a_relationship)
print(count_part_of_relationship)        


68896
6815


In [6]:
# simple test if there is same number of parents as childrens
count_children = 0
count_parents = 0
for ID, node in nodeDict.items():
    if node.children:
        count_children += len(node.children)
    if node.parents:
        count_parents += len(node.parents)

print(count_children == count_parents)
print(count_IS_a_relationship + count_part_of_relationship == count_parents)

True
True


In [7]:
# Removing regulates, negatively_regulates,positively_regulates, part_of, term_tracker_item
nodes_to_remove = ["regulates", "negatively_regulates", "positively_regulates", "part_of", "term_tracker_item"]
for node in nodes_to_remove:
    nodeDict.pop(node)

In [8]:
# creating root element
root = Node("root")
for ID, node in nodeDict.items():
    if not node.parents:
        root.children.append(node.name)
        node.parents.append(root.name)
        
nodeDict["root"] = root
numberOfNodes = len(nodeDict)

In [9]:
# adding to the dictionary alternative ID, these points to the same node as the their original ID
count_Alt_ID = 0
for ID, alt_ID_list in alternative_ID_dict.items():
    for alt_ID in alt_ID_list:
        if (alt_ID in nodeDict):
            count_Alt_ID += 1
            
        nodeDict[alt_ID] = nodeDict[ID]
if count_Alt_ID !=0:
    print("nieco je zle") # TEST if the alt_ID is already in nodeDict, if it were, my proccesing of GO would be bad

In [10]:
# check visited and change it before usage of BFS
def BFS(nodeID, nodeDict):
    current_node = nodeDict[nodeID]
    for child in current_node.children:
        if not nodeDict[child].visited:
            nodeDict[child].visited = True
            BFS(child, nodeDict)
            current_node.reachable_nodes_count += nodeDict[child].reachable_nodes_count

BFS("root", nodeDict) 
#test if bfs is valid
root.reachable_nodes_count == numberOfNodes

True

In [11]:
for node in nodeDict.values():
    node.infromationMetric = node.reachable_nodes_count / numberOfNodes

In [12]:
#just test
for i  in root.children:
    print(nodeDict[i].infromationMetric)

0.6448461502761406
0.26131247969554927
0.09381816494175524


## Clustering and metric

### Get data for clustering

In [13]:
gene_disease_csv = pd.read_csv(FOLDER+ "gene_disease_view.csv")
gene_disease_series = gene_disease_csv.groupby(["geneName"])["diseaseId"].apply(list)
gene_disease_dict = gene_disease_series.to_dict()

gene_to_list_of_diseases = []

#propably could be improved somehow with pandas functionality
for gene_name, disease_ids in gene_disease_dict.items():
    new_dict = {'diseaseId': disease_ids, 'geneName': str(gene_name)}
    gene_to_list_of_diseases.append(new_dict)

/tmp/ipykernel_1769839/1795381549.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_disease_csv = pd.read_csv(FOLDER+ "gene_disease_view.csv")


### Creating ontology

In [14]:
HUMAN = 'NCBITaxon:9606'
PART_OF = 'BFO:0000050'

ofactory = OntologyFactory()
ont = ofactory.create("GO").subontology(relations=['subClassOf', PART_OF])

afactory = AssociationSetFactory()
aset = afactory.create(ontology=ont,
                       subject_category='gene',
                       object_category='function',
                       taxon=HUMAN)

2023-05-29 17:55:01 [WARNING] [PID:1769839 TID:139769459459904] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'colocalizes_with']
2023-05-29 17:55:02 [WARNING] [PID:1769839 TID:139769459459904] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'contributes_to']
2023-05-29 17:55:02 [WARNING] [PID:1769839 TID:139769459459904] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'colocalizes_with']
2023-05-29 17:55:02 [WARNING] [PID:1769839 TID:139769459459904] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'contributes_to']
2023-05-29 17:55:02 [WARNING] [PID:1769839 TID:139769459459904] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'contributes_to']
2023-05-29 17:55:02 [WARNING] [PID:1769839 TID:139769459459904] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'contributes_to']
2023-05-29 17:55:02 [WARNING] [PID:1769839 TID:139769459459904] [golr_query.py

### Metric definition

In [15]:
def union(lst1, lst2):
    return list(set(lst1) | set(lst2))

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [16]:
def metric_based_on_information_theory(cluster):
    counter = 0
    result = 0
    size_of_cluster = len(cluster)
    for i in range(size_of_cluster):
        for j in range(i + 1, size_of_cluster):
            goTerms = intersection(ont.ancestors(cluster[i]),ont.ancestors(cluster[j]))
            minimum_infromation_metric = 1
            for goTerm in goTerms:
                infromation_metric = nodeDict[goTerm].infromationMetric
                if minimum_infromation_metric > infromation_metric:
                    minimum_infromation_metric = infromation_metric
                    
                
            result -= math.log(minimum_infromation_metric)
            counter +=1
    if counter == 0 :
        return 0
    return result/counter
            

In [17]:
def jaccard_similarity(gene1, gene2):
    ancestors1 = ont.ancestors(gene1)
    ancestors2 = ont.ancestors(gene2)
    union_genes = union(ancestors1,ancestors2)
    num_union = len(union_genes)
    if  num_union == 0:
        return 0
    return len(intersection(ancestors1,ancestors2)) / num_union

def metric_cluster_jaccard(cluster):
    counter = 0
    result = 0
    size_of_cluster = len(cluster)
    for i in range(size_of_cluster):
        for j in range(i + 1, size_of_cluster):
            result += jaccard_similarity(cluster[i],cluster[j])
            counter +=1
    if counter == 0 :
        return 0
    return result/counter
            

### Feature extraction

In [18]:
vec = DictVectorizer()
matrixGeneDisease = vec.fit_transform(gene_to_list_of_diseases)
matrixGeneDisease

<21666x51836 sparse matrix of type '<class 'numpy.float64'>'
	with 3282990 stored elements in Compressed Sparse Row format>

# Clustering

In [19]:
pickle_path = './../pickle/'
ms_filename = pickle_path + 'ms_object.pkl'
ms_labels_filename = 'ms_labels.pkl'
af_filename = pickle_path + 'af_object.pkl'
af_labels_filename = pickle_path + 'af_labels.pkl'
sc_filename = pickle_path + 'sc_object.pkl'
sc_labels_filename = pickle_path + 'sc_labels.pkl'
ac_filename = pickle_path + 'ac_object.pkl'
ac_labels_filename = pickle_path + 'ac_labels.pkl'

max_cluster_size = 1000
max_clusters_number = 1000

### Save the result of the computation

In [20]:
def save_clustering(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

def load_clustering(filename):
    with open(filename, 'rb') as f:
        labels = pickle.load(f)
    return labels

## MeanShift

In [ ]:
if os.path.exists(ms_filename) and os.path.exists(ms_labels_filename):
    result = load_clustering(ms_filename)
    ms_labels = load_clustering(ms_labels_filename)
else:
    bandwidth = estimate_bandwidth(matrixGeneDisease.toarray(), quantile=0.5)
    ms = MeanShift(bandwidth=bandwidth)
    result = ms.fit(matrixGeneDisease.toarray())
    ms_labels = result.labels_
    save_clustering(result, ms_filename)
    save_clustering(ms_labels, ms_labels_filename)

## Affinity propagation

In [21]:
if os.path.exists(af_filename) and os.path.exists(af_labels_filename):
    af = load_clustering(af_filename)
    af_labels = load_clustering(af_labels_filename)
else:
    # af = AffinityPropagation().fit(matrixGeneDisease)
    af = AffinityPropagation(damping=0.9, max_iter=500, convergence_iter=15).fit(matrixGeneDisease)
    cluster_centers_indices = af.cluster_centers_indices_
    af_labels = af.labels_
    save_clustering(af, af_filename)
    save_clustering(af_labels, af_labels_filename)

    
# Did not converge
# ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels. warnings.warn(

/home/osloup/Desktop/Muni/2nd/pv287/venv/lib64/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


## Spectral clustering

In [ ]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

if os.path.exists(sc_filename) and os.path.exists(sc_labels_filename):
    sc = load_clustering(sc_filename)
    sc_labels = load_clustering(sc_labels_filename)
else:
    max_score = -1
    optimal_clusters = 2  # minimum number of clusters
    max_clusters = max_clusters_number  # replace with the maximum number of clusters you want to consider
    scores = []

    for n_clusters in range(2, max_clusters+1):
        sc_temp = SpectralClustering(n_clusters=n_clusters, assign_labels="discretize", random_state=0).fit(matrixGeneDisease.toarray())
        label = sc_temp.labels_
        sil_coeff = silhouette_score(matrixGeneDisease.toarray(), label, metric='euclidean')
        scores.append(sil_coeff)
        print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_clusters, sil_coeff))
        if sil_coeff > max_score:
            max_score = sil_coeff
            optimal_clusters = n_clusters

    # Plotting silhouette scores
    plt.figure(figsize=(10, 6))
    plt.plot(range(2, max_clusters+1), scores, marker='o')
    plt.title('Silhouette scores for different numbers of clusters')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette score')
    plt.grid(True)
    plt.show()

    # Re-fit the spectral clustering with optimal number of clusters
    sc = SpectralClustering(n_clusters=optimal_clusters, assign_labels="discretize", random_state=0).fit(matrixGeneDisease.toarray())
    sc_labels = sc.labels_
    save_clustering(sc, sc_filename)
    save_clustering(sc_labels, sc_labels_filename)

## Agglomerative Clustering
We are trying to estimate the number of clusters

In [26]:

if os.path.exists(ac_filename) and os.path.exists(ac_labels_filename):
    ac = load_clustering(ac_filename)
    ac_labels = load_clustering(ac_labels_filename)
else:
    max_score = -1
    optimal_clusters = 2  # minimum number of clusters
    max_clusters = max_clusters_number  # replace with the maximum number of clusters you want to consider
    scores = []

    for n_clusters in range(2, max_clusters+1):
        ac_temp = AgglomerativeClustering(n_clusters=n_clusters).fit(matrixGeneDisease.toarray())
        label = ac_temp.labels_
        sil_coeff = silhouette_score(matrixGeneDisease.toarray(), label, metric='euclidean')
        scores.append(sil_coeff)
        print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_clusters, sil_coeff))
        if sil_coeff > max_score:
            max_score = sil_coeff
            optimal_clusters = n_clusters

    # Plotting silhouette scores
    plt.figure(figsize=(10, 6))
    plt.plot(range(2, max_clusters+1), scores, marker='o')
    plt.title('Silhouette scores for different numbers of clusters')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette score')
    plt.grid(True)
    plt.show()

    # Re-fit the agglomerative clustering with optimal number of clusters
    ac = AgglomerativeClustering(n_clusters=optimal_clusters).fit(matrixGeneDisease.toarray())
    ac_labels = ac.labels_
    save_clustering(ac, ac_filename)
    save_clustering(ac_labels, ac_labels_filename)

### Create arrays of geneIdentificators which ont takes , for each cluster. (information metric)

In [22]:
# Gene to GO
f = open(FOLDER + 'mapping_gene_to_go.json')
mapping_geneName_to_GO = json.load(f)
f.close()

## Function for printing metric from labels

In [23]:
from typing import List, Dict

def mapping_geneName_to_GO_terms(labels: List[int]) -> List[List[str]]:
    num_clusters = max(labels) + 1
    clustersGO = [[] for _ in range(num_clusters)]

    for i, label in enumerate(labels):
        gene_name = gene_to_list_of_diseases[i]["geneName"]
        if gene_name in mapping_geneName_to_GO:
            clustersGO[label].append(mapping_geneName_to_GO[gene_name])
            
    return clustersGO


def compute_metrics(clusters: List[List[str]]) -> Dict[int, Dict[str, float]]:
    metrics = {}
    for idx, cluster in enumerate(clusters):
        if len(cluster) >= max_cluster_size:
            print(f"Skipping cluster {idx} as it is too large.")
            continue
        info_metric = metric_based_on_information_theory(cluster)
        jaccard_metric = metric_cluster_jaccard(cluster)
        metrics[idx] = {
            "size": len(cluster),
            "information_metric": info_metric,
            "jaccard_similarity": jaccard_metric
        }
    return metrics

def print_metric(labels: List[int]) -> Dict[int, Dict[str, float]]:
    clusters = mapping_geneName_to_GO_terms(labels)
    metrics = compute_metrics(clusters)
    for cluster_idx, metric_values in metrics.items():
        print(f"\nCluster {cluster_idx}:")
        print(f"Size: {metric_values['size']}")
        print(f"Information metric: {metric_values['information_metric']}")
        print(f"Jaccard similarity metric: {metric_values['jaccard_similarity']}")
    return metrics


### Print of metrics

In [ ]:
ms_metrics = print_metric(ms_labels)
ms_metrics = compute_metrics(mapping_geneName_to_GO_terms(ms_labels))
print(af_labels[0])

In [24]:
af_labels = print_metric(af_labels)
af_labels = compute_metrics(mapping_geneName_to_GO_terms(af_labels))
print(af_labels[0])


Cluster 0:
Size: 45
Information metric: 1.206198024262404
Jaccard similarity metric: 0.08334465767537859

Cluster 1:
Size: 17
Information metric: 1.3361079452877245
Jaccard similarity metric: 0.07108344370863003

Cluster 2:
Size: 91
Information metric: 1.2994927512028838
Jaccard similarity metric: 0.07120457285347771

Cluster 3:
Size: 55
Information metric: 1.517908212364978
Jaccard similarity metric: 0.08101984641363069

Cluster 4:
Size: 1
Information metric: 0
Jaccard similarity metric: 0

Cluster 5:
Size: 95
Information metric: 1.120528195321158
Jaccard similarity metric: 0.06436905720627557

Cluster 6:
Size: 16
Information metric: 0.9742245110342688
Jaccard similarity metric: 0.05871035836791492

Cluster 7:
Size: 459
Information metric: 1.3056634880567624
Jaccard similarity metric: 0.07326477252102773

Cluster 8:
Size: 8
Information metric: 1.3519703758002026
Jaccard similarity metric: 0.08580259026687598

Cluster 9:
Size: 12
Information metric: 1.8615893706038
Jaccard similarity 